# Data Preparation

In [35]:
import pandas as pd
import csv
import matplotlib.pyplot as plt

meta = pd.read_csv(r"D:\ML\Portfolio\Projects\semantic-search\datasets\metadata-matches-pre-processed.txt")

Pandas display setting management section:

In [36]:
pd.set_option('display.float_format', lambda x: f'{x:.2f}') # Format float display to 2 decimal places
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None)

In [37]:
#pd.reset_option('display.float_format')
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_colwidth')

## 1. Data Exploration
ADD DATASET DESCRIPTION??

In [ ]:
meta.head()

Updating columns w/ more descriptive names:

In [ ]:
meta.info()

## 2. Feature Engineering

A musical composition (comp.) is defined by a combination of the composition *title* and writer *names* (as well as *ownership* % - currently out of scope).

Create two new features: canonical musical comp. (*aka __composition__*) and matched musical comp. (*aka __matches__*) - capitalisation normalised set of comp. title and writer names.

In [7]:
#meta['CAN_Comp'] = meta[['CAN_Title', 'CAN_Writers_Formatted']].agg(lambda x: ' - '.join(x.dropna()), axis=1).str.lower() # Use formatted writers. x.dropna() increases runtime; if an issue, consider df.fillna('')
meta['CAN_Comp'] = meta[['CAN_Title', 'CAN_Writers_Formatted']].fillna('').agg(' - '.join, axis=1).str.lower().str.strip(' -')

#meta['MATCHED_Comp'] = meta[['MATCHED_Title', 'MATCHED_Writer_1', 'MATCHED_Writer_2', 'MATCHED_Writer_3', 'MATCHED_Writer_4']].agg(lambda x: ' - '.join(x.dropna()), axis=1).str.lower() # x.dropna() increases runtime; if an issue, consider df.fillna('')
meta['MATCHED_Comp'] = meta[['MATCHED_Title', 'MATCHED_Writer_1', 'MATCHED_Writer_2', 'MATCHED_Writer_3', 'MATCHED_Writer_4']].fillna('').agg(' - '.join, axis=1).str.lower().str.strip(' -')

In [ ]:
meta.describe(include='all')

## 3. Data Cleaning and Validation

### 1. Remove rows w/ missing writer names in the canonical part of the dataset
Comparing 'CAN_Title' vs. 'CAN_Writers_Formatted' vs. 'CAN_Writers_Raw' counts shows that some canonical composition metadata is missing writer info. Such entries are of no use and need to be removed:

In [ ]:
if meta[meta['CAN_Writers_Formatted'].isnull()].index.equals(meta[meta['CAN_Writers_Raw'].isnull()].index):
    print('Missing formatted and raw writers are on the same rows.')

In [ ]:
# Count distinct compositions where canonical writers are NaN
meta.loc[meta[['CAN_Writers_Formatted', 'CAN_Writers_Raw']].isna().all(axis=1), 'CAN_ID'].nunique() # .any(axis=1) would work as well

In [11]:
# Remove NaN rows
meta.dropna(subset=['CAN_Writers_Formatted', 'CAN_Writers_Raw'], inplace=True) # Specifying a single writer column would work as well

### 2. Review canonical database writers: validate the difference between unique formatted and unique raw

In [12]:
# Drop and rename columns for .compare() method
writers_raw = meta['CAN_Writers_Raw']
writers_raw = writers_raw.str.replace(r' \[.*?\]', '', regex=True) # RegEx removes ' [*]'

writers_formatted = meta['CAN_Writers_Formatted']

In [ ]:
if writers_raw.compare(writers_formatted).empty:
    print(f"Composers with identical names are the source of the unique count discrepency between 'CAN_Writers_Raw' and 'CAN_Writers_Formatted'") # A strong indication this is the case. Sample values have not been reviewed.

### 3. Conflicting canonical compositions
Canonical comp. w/ more than 1 distinct CAN_ID (i.e. 'duplicate works'):

In [ ]:
# Group by CAN_Comp and create a mask for compositions with more than one unique CAN_ID
can_comp_group = meta.groupby('CAN_Comp')
can_comp_mask = can_comp_group['CAN_ID'].transform('nunique') > 1
can_comp_df = meta[can_comp_mask]

# Check if there are any compositions with more than one unique identifier
if can_comp_df.empty:
    print('Each composition has only 1 unique identifier')
else:
    print(f'{can_comp_df["CAN_Comp"].nunique()} compositions have {can_comp_df["CAN_ID"].nunique()} distinct identifiers, affecting a total of {len(can_comp_df)} rows (sample below):')

can_comp_df.drop_duplicates('CAN_ID').sort_values('CAN_Title').head() # Display a sample of the rows, sorted by CAN_Title

CAN_ID w/ more than 1 distinct canonical comp. (shouldn't be the case: reason unknown, could be alternatives titles + other unknown reasons):

In [ ]:
# Group by CAN_ID and create a mask for identifiers with more than one unique CAN_Comp
can_id_group = meta.groupby('CAN_ID')
can_id_mask = can_id_group['CAN_Comp'].transform('nunique') > 1
can_id_df = meta[can_id_mask]

# Check if there are any identifiers with more than one unique composition
if can_id_df.empty:
    print('Each comp. identifier has only 1 unique composition')
else:
    print(f'{can_id_df["CAN_ID"].nunique()} identifiers have {can_id_df["CAN_Comp"].nunique()} distinct compositions, affecting a total of {len(can_id_df)} rows (sample below):')

can_id_df.drop_duplicates('CAN_Comp').sort_values('CAN_ID').head() # Display a sample of the rows, sorted by CAN_ID

The reasons for the above seem benign (for the most part), i.e. duplicate works, film cues, etc.

Circa 3,000 total rows affected (circa 1% of total useful rows). Remove _**all**_ to streamline data preparation:

In [16]:
meta = meta[~(can_id_mask | can_comp_mask)].copy() # Remove all rows

### 4. Remove rows w/ missing matched writers
If 'MATCHED_Writer_1' is NaN all other writer columns should be NaN in the 'matched' part of the dataset. 

In [ ]:
len(meta[meta['MATCHED_Writer_1'].isna()]) # Total NaN rows

In [ ]:
matched_nan_writer_diff = len(meta[meta['MATCHED_Writer_1'].isna()]) - len(meta[meta[['MATCHED_Writer_1', 'MATCHED_Writer_2', 'MATCHED_Writer_3', 'MATCHED_Writer_4']].isna().all(axis=1)])
print(f'{matched_nan_writer_diff} rows with NaN Writer 1 have writers in other columns')

Remove all rows (incl. w/ writers in other columns, as only 23):

In [19]:
meta.dropna(subset=['MATCHED_Writer_1'], inplace=True) # Remove all rows

### 5. Conflicting matches
Matches w/ more than 1 distinct CAN_ID, i.e. potentially *incorrect* matches:

In [ ]:
# Group by MATCHED_Comp and create a mask for matches with more than one unique CAN_ID
matched_comp_group = meta.groupby('MATCHED_Comp')
matched_comp_mask = matched_comp_group['CAN_ID'].transform('nunique') > 1
matched_comp_df = meta[matched_comp_mask]

# Check if there are any distinct matches with more than one distinct CAN_ID
if matched_comp_df.empty:
    print('Each match has only 1 unique "CAN_ID"')
else:
    print(f'{matched_comp_df["MATCHED_Comp"].nunique()} matches have {matched_comp_df["CAN_ID"].nunique()} distinct compositions, affecting a total of {len(matched_comp_df)} rows (sample below):')

matched_comp_df.drop_duplicates(['CAN_ID', 'CAN_Comp']).sort_values('CAN_ID').head() # Display a sample of the rows, sorted by CAN_ID


The above conflict is due to duplicate canonical works. The most streamlined solution is to remove *all* rows:

In [21]:
meta = meta[~matched_comp_mask].copy() # Remove all rows

### 6. Canonical and matched title check


A quick additional incorrect match spot check: distinct matches where cannonical title does not equal the matched title and validation rating is 3 (not 100% sure what validation rating means, but should be the higher the rating, the more reliable match - tbc):

In [22]:
title_mask = meta['CAN_Title'].str.lower() != meta['MATCHED_Title'].str.lower()
val_mask = meta['VAL_RATING'] == 3

In [ ]:
meta[title_mask & val_mask].drop_duplicates(['CAN_ID', 'MATCHED_Comp']).sort_values('CAN_ID', ascending=False).head() # Modify n in .head(n) based on how many distinct matches to spot check

## 4. Publisher Names in Matched Writer Columns
Not part of descriptive metadata. Part of rights management metadata and is out of scope for PoC/ prototype.

1. Quick fix - only consider **MATCHED_Title** + **MATCHED_Writer_1** in a matched composition (**MATCHED_Comp_Min**)
2. Publisher names follow certain patterns (e.g. includes 'Music', 'Publishing', etc.). Find and replace publisher names w/ empty strings

Create **MATCHED_Comp_Min** feature (quick fix):

In [24]:
meta['MATCHED_Comp_Min'] = meta[['MATCHED_Title', 'MATCHED_Writer_1']].agg(' - '.join, axis=1).str.lower().str.strip(' -')

In [ ]:
meta.describe(include='all')

## 5. Distribution Analysis of *Unique Matches per CAN_ID*

### 1. Summary statistics

Mean (2.12) > median (1) indicates a positvely skewed distribution w/ most of the points concentrated on the left side (lower values) and a long tail on the right side (higher values), potentially w/ some high value outliers pulling the mean to the right. 

In [ ]:
match_count = meta.groupby('CAN_ID')['MATCHED_Comp_Min'].nunique().sort_values(ascending=False) # Create groups of CAN_IDs and their distinct matches
match_count.describe()

### 2. Drill-down analysis

Outliers:

In [ ]:
match_count[match_count > 40] # Compositions w/ top unique match counts

In [28]:
#meta[(meta['CAN_ID'] == 'C2346067')].drop_duplicates(['CAN_ID', 'MATCHED_Comp_Min']).sort_values('MATCHED_Writer_1') # Distinct match count per CAN_ID drill-down analysis
#& (~meta['MATCHED_Writer_1'].str.contains('TORM', case=False))] # Narrow down by matched writer name

### 3. Distribution of distinct matches per composition

In [ ]:
# Frequency table
match_value_count = match_count.value_counts().sort_index()
match_value_count[match_value_count > 1000].map('{:,}'.format) 

In [ ]:
plt.figure(figsize=(10, 6))  # Create a new figure with specified size
match_value_count.plot(kind='bar')  # Plot the bar chart using the frequency table

# Set plot title and axis labels
plt.title('Distribution of Matches per Composition')
plt.xlabel('Number of Distinct Matches')
plt.ylabel('Composition Count')
plt.grid(axis='y')  # Add a grid to the y-axis

plt.show()

## 6.   BERT Dataset Preparation

Format the dataset both for semantic search and fine-tuning.

In [ ]:
meta_tuning = meta[['CAN_ID', 'CAN_Title', 'CAN_Writers_Formatted', 'MATCHED_Title', 'MATCHED_Writer_1']].copy() # Use ony MATCHED_Writer_1 (omit MATCHED_Writer_2/3/4 for the PoC as contain publisher names as well writer names)

tuning_cols = meta_tuning.columns.difference(['CAN_ID'])
meta_tuning[tuning_cols] = meta_tuning[tuning_cols].apply(lambda s: s.str.lower()) # Normalise case to lower

meta_tuning.drop_duplicates(['MATCHED_Title', 'MATCHED_Writer_1'], inplace=True) # Remove repeating matches

meta_tuning.describe(include='all')

In [ ]:
meta_tuning.head()

In [ ]:
meta_tuning.shape

Export to CVS:

In [34]:
meta_tuning_export_path = r"D:\ML\Portfolio\Projects\semantic-search\datasets\data.csv"

meta_tuning.to_csv(meta_tuning_export_path, index=False, quoting=csv.QUOTE_NONNUMERIC) # Quotes to prevent Pandas converting to numeric on import to preserve the original file data types. The empty strings are converted to NaN values on import.